In [1]:
import csv
import os
import time

In [2]:
def read_csv(path):
    """
    Reads the CSV file at path, and returns a list of rows. Each row is a
    dictionary that maps a column name to a value in that column, as a string.
    """
    output = []
    for row in csv.DictReader(open(path)):
        output.append(row)
    return output


In [3]:
def row_to_edge(row):
    """
    Given an election result row or poll data row, returns the Democratic edge
    in that state.
    """
    return float(row["Dem"]) - float(row["Rep"])  

def state_edges(election_result_rows):
    """
    Given a list of election result rows, returns state edges.
    The input list does has no duplicate states;
    that is, each state is represented at most once in the input list.
    """
    dic={}
    for i in election_result_rows:
        dic.update({ i["State"]: row_to_edge(i)})
    return dic

In [4]:
def earlier_date(date1, date2):
    """
    Given two dates as strings (formatted like "Oct 06 2012"), returns True if 
    date1 is after date2.
    """
    return (time.strptime(date1, "%b %d %Y") < time.strptime(date2, "%b %d %Y"))

def most_recent_poll_row(poll_rows, pollster, state):
    """
    Given a list of poll data rows, returns the most recent row with the
    specified pollster and state. If no such row exists, returns None.
    """
    recent_date = "Oct 01 1500"
    data = None
    for i in poll_rows:
        if i["State"] == state and i["Pollster"] == pollster:
            if earlier_date(recent_date, i["Date"]):
                recent_date = i["Date"]
                data = i
    return data


In [5]:
def unique_column_values(rows, column_name):
    """
    Given a list of rows and the name of a column (a string), returns a set
    containing all values in that column.
    """
    my_set = set()
    for i in rows:
        if i[column_name] not in my_set:
            my_set.add(i[column_name])
    return my_set

def pollster_predictions(poll_rows):
    """
    Given a list of poll data rows, returns pollster predictions.
    """
    dic = {}
    Pollster_set = unique_column_values(poll_rows, "Pollster")
    State_set = unique_column_values(poll_rows, "State")
    for i in Pollster_set:
        temp = []
        for j in State_set:
            a = most_recent_poll_row(poll_rows, i, j)
            if a is not None:
                temp.append(a)
                b = state_edges(temp)
                dic[i] = b            
    return dic

In [6]:
def average_error(state_edges_predicted, state_edges_actual):
    """
    Given predicted state edges and actual state edges, returns
    the average error of the prediction.
    """
    error = 0
    count = 0
    predict_set = set(state_edges_predicted)
    actual_set = set(state_edges_actual)
    for name in predict_set.intersection(actual_set):
        error += abs(state_edges_predicted[name] - state_edges_actual[name])
        count += 1
    return (error/count)

def pollster_errors(pollster_predictions, state_edges_actual):
    """
    Given pollster predictions and actual state edges, retuns pollster errors.
    """
    dic = {}
    for innerDict in pollster_predictions:                        ###innerDict -> Pollster
        Pollster_predict = pollster_predictions[innerDict]        ###{'WA': 1.0, 'CA': -2.0, 'ID': -20.0}
        avg_err = average_error(Pollster_predict, state_edges_actual)
        dic.update({innerDict: avg_err})
    return dic

In [7]:
def pivot_nested_dict(nested_dict):
    """
    Pivots a nested dictionary, producing a different nested dictionary
    containing the same values.
    The input is a dictionary d1 that maps from keys k1 to dictionaries d2,
    where d2 maps from keys k2 to values v.
    The output is a dictionary d3 that maps from keys k2 to dictionaries d4,
    where d4 maps from keys k1 to values v.
    For example:
      input = { "a" : { "x": 1, "y": 2 },s
                "b" : { "x": 3, "z": 4 } }
      output = {'y': {'a': 2},
                'x': {'a': 1, 'b': 3},
                'z': {'b': 4} }
    """

    from collections import defaultdict
    out = defaultdict(dict)
    for outer_key, inner in nested_dict.items():
        for inner_key, value in inner.items():
            out[inner_key][outer_key] = value
    return out

In [8]:
def average_error_to_weight(error):
    """
    Given the average error of a pollster, returns that pollster's weight.
    The error must be a positive number.
    """
    return error ** (-2)

# The default average error of a pollster who did no polling in the
# previous election.
DEFAULT_AVERAGE_ERROR = 5.0


In [9]:
def pollster_to_weight(pollster, pollster_errors):
    """"
    Given a pollster and a pollster errors, return the given pollster's weight.
    """
    if pollster not in pollster_errors:
        weight = average_error_to_weight(DEFAULT_AVERAGE_ERROR)
    else:
        weight = average_error_to_weight(pollster_errors[pollster])
    return weight

In [10]:
def weighted_average(items, weights):
    """
    Returns the weighted average of a list of items.
    
    Arguments:
    items is a list of numbers.
    weights is a list of numbers, whose sum is nonzero.
    
    Each weight in weights corresponds to the item in items at the same index.
    items and weights must be the same length.
    """
    assert len(items) > 0
    assert len(items) == len(weights)
    total = 0
    sum_weights = 0
    for i in range(len(items)):
        total += items[i] * weights[i]
        sum_weights += weights[i]
    return total/sum_weights


In [11]:
def average_edge(pollster_edges, pollster_errors):
    """
    Given pollster edges and pollster errors, returns the average of these edges
    weighted by their respective pollster errors.
    """
    from decimal import Decimal
    weight_sum = 0
    denum = 0
    weight = 0
    for i in pollster_edges:
        weight = Decimal(pollster_to_weight(i, pollster_errors))
        weight_sum += Decimal(weight)
        denum += Decimal(weight)*Decimal(pollster_edges[i]) 
    result = float(denum)/float(weight_sum)
    return result